In [11]:
import urllib
import requests
import json
import pandas as pd
from pandas import read_sql
import datetime
import matplotlib.pyplot as plt
import os
import numpy as np
from copy import deepcopy
import pymysql as mdb
from sklearn.cluster import KMeans

In [3]:
afile="/Users/abramvandergeest/mysql_insightwiki_auth.txt"
a=open(afile)
passwd=a.readline().rstrip()
a.close()
host='localhost'; user='abram.ghost';db='wikidata2011'
con = mdb.connect(host, user, passwd, db)

numtopics=76

In [4]:
#def week_list(base):
#    numdays=7
#    return([base - datetime.timedelta(days=x) for x in range(0, numdays)])

def datefromstr(str):
    return datetime.datetime(int(str[0:4]), int(str[5:7]),int(str[8:]))

str='2015091618'
def date_parse(str):
    year =int(str[:4])
    month=int(str[4:6])
    day=int(str[6:8])
    hour=int(str[8:])
    return datetime.datetime(year,month,day,hour)

def datetime2str(t):
    return "%4d%02d%02d%02d"%(t.year,t.month,t.day,t.hour)
    
def gen_feat(into):
    avg=np.average(into)
    if avg==0: avg=1
    maxv= np.max(into)/avg
    minv= np.min(into)/avg
    rmsv= np.std(into)/avg
    difxv=np.max(np.diff(into))/avg
    difnv=np.min(np.diff(into))/avg
    return([maxv,minv,rmsv,difxv,difnv])


In [5]:
with con:
    curt= con.cursor()
    #sql="SELECT COUNT(*) FROM `topics` "
    
    sql="SELECT `Id`,`topic_label`,`topic_string` FROM `topics`;"
    curt.execute(sql)
    topics=[]
    for topic in curt:
        topics.append(topic)

data={}

df=range(numtopics)
with con:
    curt= con.cursor()
    #sql="SELECT COUNT(*) FROM `topics` "
    #curt.execute(sql)
    sql="SELECT `Id`,`topic_label`,`topic_string` FROM `topics`;"
    curt.execute(sql)
    for row in curt:
        #cur = con.cursor()
        sql='''SELECT `page_views`.`count` AS `vc`,`page_views`.`date` AS `vd` 
             FROM `topics` INNER JOIN `page_views` on `topics`.`ID` = `page_views`.`topic_id` 
             WHERE `topics`.`id`=%s '''

        #print sql
        
        data[row[1]]=read_sql(sql, con,params=[row[0]])
        df[row[0]]=data[row[1]]
        #data[row[1]]=df[len(df)-1][1]

topicdata=df

In [6]:
timeforcluster=datetime.datetime(2011,10,10)

In [7]:
##for top in topics:
#top=topics[1]
#for top in topics:
#    topicdata[top[0]]['date']=map(lambda x:date_parse(x),data[top[1]]['vd'] )
    
    
topicdata[1]['vc'].head(10)


0    2298
1    2589
2    1438
3    2138
4    2476
5    2125
6    1925
7    2018
8    2174
9    1799
Name: vc, dtype: int64

In [8]:
for i in xrange(9,13):
    print topics[i][1]


michelebachmann
hermancain
occupymovement
socialsecurityunitedstates


In [26]:
dateinput=datetime.datetime(2011,6,5,0)

dateinit=dateinput
datefin=dateinit+datetime.timedelta(days=7)
di=datetime2str(dateinit)
df=datetime2str(datefin)

clusinp=range(numtopics+1)
print numtopics
clusinp[0]=gen_feat([0,0,0,0,0])
clusinp=[]
clusinp.append([9,9,9,9,9])

for index,topic in enumerate(topics):
    print topic
    #topic=list(topics[index])
    d=topicdata[topic[0]]
    #print topic[1],topic[0]

    #print di ,df
    #intofeat=d[ (d['vd']>di) & (d['vd']<df )]['vc'].values
    #print intofeat
    clusinp.append(gen_feat(d[ (d['vd']>di) & (d['vd']<df )]['vc'].values))

clusinp=np.array(clusinp)
print np.shape(clusinp)
print clusinp #outputting random topic values to see the data


76
(1, 'barackobama', 'Barack Obama')
(2, 'mittromney', 'Mitt Romney')
(3, 'ronpaul', 'Ron Paul')
(4, 'fredkarger', 'Fred Karger')
(5, 'newtgingrich', 'Newt Gingrich')
(6, 'ricksantorum', 'Rick Santorum')
(7, 'buddyroemer', 'Buddy Roemer')
(8, 'rickperry', 'Rick Perry')
(9, 'johnhuntsman', 'John Huntsman')
(10, 'michelebachmann', 'Michele Bachmann')
(11, 'hermancain', 'Herman Cain')
(12, 'occupymovement', 'Occupy movement')
(13, 'socialsecurityunitedstates', 'Social Security United States')
(14, 'medicareunitedstates', 'Medicare United States')
(15, 'medicaid', 'Medicaid')
(16, 'regulation', 'Regulation')
(17, 'teapartymovement', 'Tea Party movement')
(18, 'legalitycannabis', 'Legality cannabis')
(19, 'marijuana', 'marijuana')
(20, 'russiaukraine', 'Russia Ukraine')
(21, 'libya', 'Libya')
(22, 'benghazi', 'Benghazi')
(23, 'immigra', 'Immigra')
(24, 'illegalimmigraunitedstates', 'illegal immigra united states')
(25, 'gunlaw', 'Gun law')
(26, 'guncontrol', 'Gun control')
(27, 'arabspring

In [17]:
kmeans=KMeans(n_clusters=3)
kmeans.fit(clusinp)

ValueError: setting an array element with a sequence.